## Homework 4

This assignment focuses on methods of classification when data has a class imbalance. You will compare results obtained with Synthetic Minority Oversampling, ADASYN with FastKDE, and Normalizing Flows to oversample the minority class. To develop the data application, you will use



https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction.



This dataset is very large, and in this project, you may subset 10% of observations from each class. Your results should include confusion matrices and stratified K-fold validated estimates of accuracy and recalls.

In [17]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import fastkde
import numpy as np
from sklearn.model_selection import train_test_split



#### This code should not be run again. It was used to subset the original data.

In [4]:
# importing train data
train = pd.read_csv('data/train.csv')
train

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595207,1488013,0,3,1,10,0,0,0,0,0,...,4,1,9,6,0,1,1,0,1,1
595208,1488016,0,5,1,3,0,0,0,0,0,...,4,1,3,8,1,0,1,0,1,1
595209,1488017,0,1,1,10,0,0,1,0,0,...,3,2,2,6,0,0,1,0,0,0
595210,1488021,0,5,2,3,1,0,0,0,1,...,4,1,4,2,0,1,1,1,0,0


In [ ]:
#subsetting the train data to only be 10% of each class and saving it over the original data
#subset_train = train.groupby('target').sample(frac=0.1)
#subset_train['target'].value_counts()
#subset_train.to_csv('data/train.csv', index=False)
#subset_train

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
211336,527987,0,2,1,8,0,0,0,0,0,...,3,1,1,7,1,1,0,1,0,0
18955,47519,0,3,1,3,0,0,1,0,0,...,3,2,3,5,0,0,1,1,1,1
375516,938513,0,1,1,1,0,0,1,0,0,...,1,1,1,4,0,1,1,0,0,1
111764,279774,0,0,1,2,0,0,1,0,0,...,6,0,5,3,0,1,1,0,0,0
92938,232653,0,3,1,2,1,0,0,1,0,...,4,0,5,11,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483055,1207113,1,1,1,7,0,4,0,0,1,...,3,1,1,9,0,0,1,0,1,0
485292,1212728,1,0,1,5,0,0,0,1,0,...,6,2,1,2,0,1,1,0,0,0
303247,757821,1,3,1,7,0,1,1,0,0,...,3,1,3,8,0,0,0,1,0,0
350300,875118,1,0,3,5,1,0,0,1,0,...,7,2,4,6,0,1,1,0,0,0


## ADASYN with FastKDE

In [52]:
# reimporting data to make sure that subset is used
subset_train = pd.read_csv('data/train.csv')

In [53]:
# spliting into X and y
X = subset_train.drop(columns=['target'])
y = subset_train['target']

# Convert X and y to numpy arrays
X = np.array(X)  # Convert X to a numpy array if it's a pandas DataFrame
y = np.array(y) 

In [58]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import ADASYN
from fastkde import fastkde
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score
from sklearn.ensemble import RandomForestClassifier

# converting X and y to numpy arrays
X = np.array(X) 
y = np.array(y) 

# applying FastKDE to the feature data to smooth and estimate the density
num_points = 257  # setting number of points for KDE
var_names = [f'feature_{i}' for i in range(X.shape[1])]  # assigning names to each feature

# calculating the pdf for each feature in the data, the density estimation for each feature is stored in kde_result
kde_result = {}
for i in range(X.shape[1]):
    kde_result[f'feature_{i}'] = fastkde.pdf(X[:, i], var_names=[var_names[i]], num_points=num_points)

# using ADASYN to generate synthetic data to balance the data
adasyn = ADASYN(sampling_strategy='minority', n_neighbors=5)
X_resampled, y_resampled = adasyn.fit_resample(X, y)

# splitting the resampled data using stratified k-fold
skf = StratifiedKFold(n_splits=5, shuffle=True)

# creating lists to store scores and confusion matrices
accuracy_scores = []
recall_scores = []
conf_matrices = []

# iterating through each fold, training a model, and evaluating it
for train_index, test_index in skf.split(X_resampled, y_resampled):
    X_train, X_test = X_resampled[train_index], X_resampled[test_index]
    y_train, y_test = y_resampled[train_index], y_resampled[test_index]

    # training Random Forest model on training data
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # making predictions on the test data
    y_pred = model.predict(X_test)

    # evalutating model performance
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred, average='binary'))
    conf_matrices.append(confusion_matrix(y_test, y_pred))

# printing average scores across all folds
print("Average Accuracy: ", np.mean(accuracy_scores))
print("Average Recall: ", np.mean(recall_scores))
print("Confusion Matrix (average across folds):\n", np.mean(conf_matrices, axis=0))


Average Accuracy:  0.9809060211375698
Average Recall:  0.9615799132689758
Confusion Matrix (average across folds):
 [[11470.4     0. ]
 [  435.4 10897.2]]


In [59]:
def crossvalidate(kf, x, y, model):
    cm_list = []
    accuracy = []
    recall = []
    for train_index, test_index in kf.split(x, y):
        xtrain, xtest = x[train_index], x[test_index]
        ytrain, ytest = y[train_index], y[test_index]
        model.fit(xtrain, ytrain)
        y_pred = model.predict(xtest)
        accuracy.append(accuracy_score(ytest, y_pred))
        recall.append(recall_score(ytest, y_pred, average = 'binary'))
        cm_list.append(confusion_matrix(ytest, y_pred))
    return cm_list, accuracy, recall

In [60]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import ADASYN
from fastkde import fastkde
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score
from sklearn.ensemble import RandomForestClassifier

# converting X and y to numpy arrays
X = np.array(X) 
y = np.array(y) 

# applying FastKDE to the feature data to smooth and estimate the density
num_points = 257  # setting number of points for KDE
var_names = [f'feature_{i}' for i in range(X.shape[1])]  # assigning names to each feature

# calculating the pdf for each feature in the data, the density estimation for each feature is stored in kde_result
kde_result = {}
for i in range(X.shape[1]):
    kde_result[f'feature_{i}'] = fastkde.pdf(X[:, i], var_names=[var_names[i]], num_points=num_points)

# using ADASYN to generate synthetic data to balance the data
adasyn = ADASYN(sampling_strategy='minority', n_neighbors=5)
X_resampled, y_resampled = adasyn.fit_resample(X, y)

# splitting the resampled data using stratified k-fold
skf = StratifiedKFold(n_splits=5, shuffle=True)

# creating the RandomForest model
model = RandomForestClassifier()

# calling the crossvalidate function with the resampled data
cm_list, accuracy, recall = crossvalidate(skf, X_resampled, y_resampled, model)

# printing average scores across all folds
print("Average Accuracy: ", np.mean(accuracy))
print("Average Recall: ", np.mean(recall))
print("Confusion Matrix (average across folds):\n", np.mean(cm_list, axis=0))


Average Accuracy:  0.9809147919133447
Average Recall:  0.9615975281430744
Confusion Matrix (average across folds):
 [[11470.4     0. ]
 [  435.2 10897.4]]
